In [21]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [22]:
house_dict = []

for page in range(1, 21):
    # Filter Out Coming Soon, Auction, Home Type: Multi-family, Days On Zillow: 90 Days
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.93471906436687,"east":-107.64370343936687,"south":31.980790552461904,"north":48.0418909706512},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false},"isCondo":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
#     user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
#     print(soup)

    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('address', class_="list-card-addr").text
#             state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
#             loop_response = requests.get(house_website, headers=headers)
#             loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
#             loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
#             loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
            
#             lot = 'null';
#             house_type = 'null';
#             year_build = 'null';
#             parking = 'null';
#             hoa = 'null';
#             lot_conversion_to_sqft = 'null';
#             zillow_price_per_sqft = 'null';
            
#             for content in loop_result2:
#                 if "Lot" in content.text.split(":")[0]:
#                     lot = content.text.split(":")[1];
#                 if "Type" in content.text.split(":")[0]:
#                     house_type = content.text.split(":")[1];
#                 if "Year built" in content.text.split(":")[0]:
#                     year_build = content.text.split(":")[1];
#                 if "Parking" in content.text.split(":")[0]:
#                     parking = content.text.split(":")[1];
#                 if "HOA" in content.text.split(":")[0]:
#                     hoa = content.text.split(":")[1];
#                 if "Price/sqft" in content.text.split(":")[0]:
#                     zillow_price_per_sqft = content.text.split(":")[1];
                    
#             try:
#                 if (str(lot.split(" ")[1]) == "sqft"):
#                     lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
#                 elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
#                     lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
#                 elif (lot == "No Data"):
#                     lot_conversion_to_sqft = 'null'
#                 else:
#                     lot_conversion_to_sqft = 'null'
#             except IndexError:
#                 lot_conversion_to_sqft = 'null'
                
#             try:
#                 zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-19crqy3-25 sc-1wz5zm-8 fSMRgz")[0].text
#                 if (zestimate == "Zestimate®"):
#                     zestimate = 'null'
# #                 print('zestimate', zestimate)
#                 rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-19crqy3-25 sc-1wz5zm-8 fSMRgz")[1].text
#             except:
#                 zestimate = 'null'
# #                 print('zestimate', zestimate)
#                 rent_zestimate = 'null'
# #             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": "Multi Family",
#                 "Year_build": year_build,
#                 "Parking": parking,
#                 "HOA": hoa,
#                 "Lot": lot,
#                 "Lot_conversion_to_sqft": lot_conversion_to_sqft,
#                 "Zestimate": zestimate,
#                 "Rent_zestimate": rent_zestimate,
#                 "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(2)
            
        except AttributeError as e:
            continue

print(house_dict)

[{'Listing_type': 'For sale by owner', 'Price': '$450,000', 'Full_address': '20961 83rd St, California City, CA 93505', 'Street': '20961 83rd St', 'City': 'CaliforniaCity', 'State': 'CA', 'Zip_code': '93505', 'Bed': '12', 'Bath': '12', 'Square_Feet': '6,032', 'House_type': 'Multi Family', 'House_website': 'https://www.zillow.com/homedetails/20961-83rd-St-California-City-CA-93505/2103136090_zpid/'}, {'Listing_type': 'Apartment for sale', 'Price': '$475,000', 'Full_address': '1776 E Victoria Ave, San Bernardino, CA 92408', 'Street': '1776 E Victoria Ave', 'City': 'SanBernardino', 'State': 'CA', 'Zip_code': '92408', 'Bed': '7', 'Bath': '4', 'Square_Feet': '3,538', 'House_type': 'Multi Family', 'House_website': 'https://www.zillow.com/homedetails/1776-E-Victoria-Ave-San-Bernardino-CA-92408/17364199_zpid/'}, {'Listing_type': 'Apartment for sale', 'Price': '$449,000', 'Full_address': '66164 Desert View Ave, Desert Hot Springs, CA 92240', 'Street': '66164 Desert View Ave', 'City': 'DesertHotS

In [23]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,House_type,House_website
0,For sale by owner,"$450,000","20961 83rd St, California City, CA 93505",20961 83rd St,CaliforniaCity,CA,93505,12,12,"6,032",Multi Family,https://www.zillow.com/homedetails/20961-83rd-...
1,Apartment for sale,"$475,000","1776 E Victoria Ave, San Bernardino, CA 92408",1776 E Victoria Ave,SanBernardino,CA,92408,7,4,"3,538",Multi Family,https://www.zillow.com/homedetails/1776-E-Vict...
2,Apartment for sale,"$449,000","66164 Desert View Ave, Desert Hot Springs, CA ...",66164 Desert View Ave,DesertHotSprings,CA,92240,5,4,"2,330",Multi Family,https://www.zillow.com/homedetails/66164-Deser...
3,Foreclosure,"$375,000","1927 E 111th St, Los Angeles, CA 90059",1927 E 111th St,LosAngeles,CA,90059,6,3,"2,253",Multi Family,https://www.zillow.com/homedetails/1927-E-111t...
4,Apartment for sale,"$150,000","25450 Jade Rd, Barstow, CA 92311",25450 Jade Rd,Barstow,CA,92311,5,3,"2,917",Multi Family,https://www.zillow.com/homedetails/25450-Jade-...
...,...,...,...,...,...,...,...,...,...,...,...,...
475,Apartment for sale,"$1,385,000","120 E Main St, Santa Paula, CA 93060",120 E Main St,SantaPaula,CA,93060,6,3,"3,774",Multi Family,https://www.zillow.com/homedetails/120-E-Main-...
476,Apartment for sale,"$550,000","564 W 21st St, San Bernardino, CA 92405",564 W 21st St,SanBernardino,CA,92405,6,4,"2,614",Multi Family,https://www.zillow.com/homedetails/564-W-21st-...
477,Apartment for sale,"$1,525,000","1896 Orange Ave, Costa Mesa, CA 92627",1896 Orange Ave,CostaMesa,CA,92627,3,3,"1,714",Multi Family,https://www.zillow.com/homedetails/1896-Orange...
478,Apartment for sale,"$12,600,000","2400 Palos Verdes Dr W, Palos Verdes Peninsula...",2400 Palos Verdes Dr W,PalosVerdesPeninsula,CA,90274,8,6,"23,361",Multi Family,https://www.zillow.com/homedetails/2400-Palos-...


In [24]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q2-2020_Multi_Family_Data_page1-20.csv", index=False, header=True)